In [20]:
%pip install sqlite-vec


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
import sqlite3
import sqlite_vec

db = sqlite3.connect("memory-test-vector.db")
db.enable_load_extension(True)
sqlite_vec.load(db)
db.enable_load_extension(False)

(vec_version,) = db.execute("select vec_version()").fetchone()
print(f"vec_version={vec_version}")

vec_version=v0.1.6


In [13]:
from sqlite_vec import serialize_float32

embedding = [0.1, 0.2, 0.3, 0.4]
result = db.execute("select vec_length(?)", [serialize_float32(embedding)])

print(result.fetchone()[0])  # 4

4


In [14]:
from typing import List
import struct


def serialize_f32(vector: List[float]) -> bytes:
    """serializes a list of floats into a compact "raw bytes" format"""
    return struct.pack("%sf" % len(vector), *vector)

# memory sqlite db
#db = sqlite3.connect(":memory:")
#db.enable_load_extension(True)
#sqlite_vec.load(db)
#db.enable_load_extension(False)

sqlite_version, vec_version = db.execute(
    "select sqlite_version(), vec_version()"
).fetchone()
print(f"sqlite_version={sqlite_version}, vec_version={vec_version}")

items = [
    (1, [0.1, 0.1, 0.1, 0.1]),
    (2, [0.2, 0.2, 0.2, 0.2]),
    (3, [0.3, 0.3, 0.3, 0.3]),
    (4, [0.4, 0.4, 0.4, 0.4]),
    (5, [0.5, 0.5, 0.5, 0.5]),
]
query = [0.3, 0.3, 0.3, 0.3]

db.execute("CREATE VIRTUAL TABLE vec_items USING vec0(embedding float[4])")

with db:
    for item in items:
        db.execute(
            "INSERT INTO vec_items(rowid, embedding) VALUES (?, ?)",
            [item[0], serialize_f32(item[1])],
        )

rows = db.execute(
    """
      SELECT
        rowid,
        distance
      FROM vec_items
      WHERE embedding MATCH ?
      ORDER BY distance
      LIMIT 3
    """,
    [serialize_f32(query)],
).fetchall()

print(rows)

sqlite_version=3.41.1, vec_version=v0.1.6


OperationalError: table vec_items already exists

In [4]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv("./env/.env"))

import dashscope
from http import HTTPStatus
from pprint import pprint

with open("来自 Google 内部的另外一种声音：AI 没有护城河.md", 'r') as f:
    text = f.read()

resp = dashscope.TextEmbedding.call(
    model=dashscope.TextEmbedding.Models.text_embedding_v2,
    input="We are lucky to live in an age in which we are still making discoveries.",
    dimension=1536,
)
pprint("ok") if resp.status_code == HTTPStatus.OK else print(resp)
pprint(resp["output"]["embeddings"][0]["embedding"][:3])

'ok'
[0.022378576171554372, -0.027432455162420308, -0.00355793080956962]


In [1]:
%pip install tiktoken


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [48]:
from itertools import islice
import tiktoken
import numpy as np

def batched(iterable, n):
    """将数据分批处理成每批长度为 n 的元组。最后一批的长度可能较短。"""
    # batched('ABCDEFG', 3) --> ABC DEF G
    if n < 1:
        raise ValueError("n must be at least one")
    it = iter(iterable)
    while batch := islice(it, n):
        yield batch


def chunked_tokens(text, encoding_name, chunk_length):
    encoding = tiktoken.get_encoding(encoding_name)
    tokens = encoding.encode(text)
    chunk_items_len = len(tokens) // chunk_length + 1
    print("chunk items length: ", chunk_items_len)
    text_len = len(text) // chunk_items_len + 1
    chunks_iterator = [text[i : i + text_len] for i in range(0, len(text), text_len)]
    yield from chunks_iterator


def len_safe_get_embedding(
    text,
    model="text-embedding-v2",
    max_tokens=2048,
    encoding_name="cl100k_base",
):
    chunk_embeddings = []
    chunk_lens = []
    for chunk in chunked_tokens(
        text, encoding_name=encoding_name, chunk_length=max_tokens
    ):
        print("length: ", len(chunk))
        chunk_embeddings.append(get_embedding(chunk, model=model))
        chunk_lens.append(len(chunk))

    chunk_embeddings = np.average(chunk_embeddings, axis=0, weights=chunk_lens)
    return chunk_embeddings

def get_embedding(text, model):
    resp = dashscope.TextEmbedding.call(
        model=model,
        input=text,
        dimension=1024,
    )
    return resp["output"]["embeddings"][0]["embedding"]


print(len(text))
tokens = tiktoken.get_encoding("cl100k_base").encode(text)
print(len(tokens))
chunked_embeddings = len_safe_get_embedding(text)
print(len(chunked_embeddings))

18182
5457
chunk items length:  3
length:  6061
length:  6061
length:  6060
1536


In [25]:
print(len(text))
resp_v3 = dashscope.TextEmbedding.call(
    model=dashscope.TextEmbedding.Models.text_embedding_v3,
    input=text,
    dimension=1024,
)
print(resp_v3["output"]["embeddings"][0]['embedding'][:5])
embeddings = resp_v3["output"]["embeddings"]
print(len(embeddings))

# resp_batched = dashscope.BatchTextEmbedding.call(
#    model=dashscope.BatchTextEmbedding.Models.text_embedding_async_v2,
#    url="https://help-static-aliyun-doc.aliyuncs.com/file-manage-files/zh-CN/20241016/nigwvr/text_embedding_file.txt",
#    text_type="document"
# )
# pprint(resp_batched)

18182
[-0.07153698801994324, -0.04376567527651787, -0.030815014615654945, -0.006778706330806017, -0.06811530888080597]
1


In [26]:
print(embeddings[0]["embedding"][:5])


[-0.07153698801994324, -0.04376567527651787, -0.030815014615654945, -0.006778706330806017, -0.06811530888080597]


In [18]:
db.execute(
    """CREATE VIRTUAL TABLE if NOT EXISTS vec_document_items USING vec0(
    id INTEGER PRIMARY KEY,
    document_name TEXT,
    embedding float[1024]
)"""
)

id = 1
name = "./vectordb/来自 Google 内部的另外一种声音：AI 没有护城河.md"
vector = embeddings[0]["embedding"]

db.execute(
    "INSERT INTO vec_document_items(id, document_name, embedding) VALUES (?, ?, ?)",
    [id, name, serialize_f32(vector)],
)
db.commit()

In [45]:
db.execute(
    """CREATE VIRTUAL TABLE if NOT EXISTS vec_chunked_document_items USING vec0(
    id INTEGER PRIMARY KEY,
    document_name TEXT,
    embedding float[1536]
)"""
)
with db:
    for id, embedding in enumerate(chunked_embeddings):
        db.execute(
            "INSERT INTO vec_chunked_document_items(id, document_name, embedding) VALUES (?, ?, ?)",
            [id, name, serialize_f32(embedding)],
        )

In [46]:
# insert again
with db:
    for id, embedding in enumerate(chunked_embeddings):
        db.execute(
            "INSERT INTO vec_chunked_document_items(id, document_name, embedding) VALUES (?, ?, ?)",
            [id, name, serialize_f32(embedding)],
        )

OperationalError: UNIQUE constraint failed on vec_chunked_document_items primary key

In [12]:
%pip install langchain-text-splitters sentence-transformers sqlite-vss

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 3.3 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.embeddings.dashscope import DashScopeEmbeddings
from langchain_community.vectorstores import SQLiteVSS
from langchain_text_splitters import CharacterTextSplitter, MarkdownHeaderTextSplitter
from dotenv import load_dotenv, find_dotenv


# init qwen model config
_ = load_dotenv(find_dotenv("./env/.env"))

# load the document and split it into chunks
loader = TextLoader("./来自 Google 内部的另外一种声音：AI 没有护城河.md")
documents = loader.load()
with open("./来自 Google 内部的另外一种声音：AI 没有护城河.md", "r") as f:
    md_str = f.read()

# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
headers_to_split_on = [("#", "Header 1"), ("##", "Header 2"), ("###", "Header 3")]
markdown_text_splitter = MarkdownHeaderTextSplitter(headers_to_split_on)
docs = text_splitter.split_documents(documents)
md_docs = markdown_text_splitter.split_text(md_str)
texts = [doc.page_content for doc in docs]
md_texts = [md_doc.page_content for md_doc in md_docs]
print(md_docs[1])


# create the open-source embedding function
# embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
embedding_function = DashScopeEmbeddings(model="text-embedding-v3")


# load it in sqlite-vss in a table named state_union.
# the db_file parameter is the name of the file you want
# as your sqlite database.
db = SQLiteVSS.from_texts(
    texts=md_texts,
    embedding=embedding_function,
    table="state_union",
    db_file="./vss.db",
)

# query it
query = "AI的壁垒没有想象中的那么高"
data = db.similarity_search(query)

# print results
data[0].page_content

page_content='最近读到一篇去年泄漏的Google内部研究员的文件，要提前说明的是这份文件中的观点从现在来看是有待商榷的，但是它确实提供了另外一个观察这波 AI 浪潮的视角，让个人开发者甚至中小企业思考如何参与这一次技术革新。' metadata={'Header 1': '来自 Google 内部的另外一种声音：AI 没有护城河'}


'行业数据（如安全、金融、医疗）具有高度的专业性且难以获取，形成了一定的数据壁垒，高质量的数据标注需要大量的人力物力，这使得别人难以追赶。同时优秀的 AI 算法往往涉及复杂的数学模型和优化技术，需要高水平的人才，而对特定场景的模型微调需要大量的实验和经验积累，从而导致AI人才供不应求。最后AI模型训练需要大量的计算资源，需要更强大的算力基础设施。  \n与此同时，TensorFlow、PyTorch 等开源框架降低了 AI 开发的门槛，使得技术更容易被复制。预训练模型的共享加速了模型开发，缩短了模型训练时间。 AI 领域技术更新迭代迅速，算法的领先优势可能很快被超越。云计算的发展降低了算力获取的成本，使得获得足够的算力的门槛降低。越来越严格的数据隐私法规限制了数据的获取和利用，削弱了数据壁垒。  \n正式这样复杂的优势与局限导致：\n- 强者恒强： 拥有雄厚资金实力的科技巨头将在AI竞争中占据主导地位。\n- 同质化竞争加剧： 各家公司推出的AI产品同质化严重，差异化竞争不足。\n- 硬件受限： AI硬件的核心技术被少数几家公司垄断，中小公司难以在硬件领域取得突破。'

In [4]:
db.similarity_search_with_score(query)

[(Document(metadata={}, page_content='行业数据（如安全、金融、医疗）具有高度的专业性且难以获取，形成了一定的数据壁垒，高质量的数据标注需要大量的人力物力，这使得别人难以追赶。同时优秀的 AI 算法往往涉及复杂的数学模型和优化技术，需要高水平的人才，而对特定场景的模型微调需要大量的实验和经验积累，从而导致AI人才供不应求。最后AI模型训练需要大量的计算资源，需要更强大的算力基础设施。  \n与此同时，TensorFlow、PyTorch 等开源框架降低了 AI 开发的门槛，使得技术更容易被复制。预训练模型的共享加速了模型开发，缩短了模型训练时间。 AI 领域技术更新迭代迅速，算法的领先优势可能很快被超越。云计算的发展降低了算力获取的成本，使得获得足够的算力的门槛降低。越来越严格的数据隐私法规限制了数据的获取和利用，削弱了数据壁垒。  \n正式这样复杂的优势与局限导致：\n- 强者恒强： 拥有雄厚资金实力的科技巨头将在AI竞争中占据主导地位。\n- 同质化竞争加剧： 各家公司推出的AI产品同质化严重，差异化竞争不足。\n- 硬件受限： AI硬件的核心技术被少数几家公司垄断，中小公司难以在硬件领域取得突破。'),
  0.5443615913391113),
 (Document(metadata={}, page_content='尽管 Google 研究员认为 AI 行业强者恒强、同质化竞争剧烈，但仍可以从以下几个方面探讨如何建立潜在的护城河，特别是对个人和中小企业而言：\n- 数据质量： 优质、标注精细的数据是训练AI模型的关键。拥有高质量数据的公司可能具备一定优势。 专注于特定行业，积累行业知识和数据，形成深厚的技术壁垒。\n- 工程创新： 虽然核心算法是开源的，但对算法的工程优化和创新仍然是提升模型性能的重要途径。 持续投入研究，不断突破提升使用体验也可以保持领先。\n- Know-How： 将 AI 技术与特定行业深度结合，积累行业Know-How，可以形成一定的竞争壁垒。\n- 人才团队： 拥有优秀的 AI 人才团队，能够持续进行创新和研发，也是一种护城河。'),
  0.6289268732070923),
 (Document(metadata={}, page_content='AI 行业是一个典型的完全竞争行业，

In [ ]:
import tiktoken

MODEL_2_ENCODING = {"text-embedding-v1": "cl100k_base", "text-embedding-v2": "cl100k_base", "text-embedding-v3": "cl100k_base"}
MAX_TOKEN = {"text-embedding-v1": 2048, "text-embedding-v2": 2048, "text-embedding-v3":8192}

model_name = "text-embedding-v2"
encoding = tiktoken.get_encoding(MODEL_2_ENCODING[model_name])
tokens = encoding.encode(md_str)
print(f"{len(tokens)} tokens")
print(tokens[0], tokens[1])

if len(tokens) > MAX_TOKEN[model_name]:
    print(f"{len(tokens)} tokens exceeds the maximum of {MAX_TOKEN[model_name]} tokens")
    print(f"{len(tokens)} tokens need to chunk")

5457 tokens
11192 2150
5457 tokens exceeds the maximum of 2048 tokens
5457 tokens need to chunk


In [2]:
# retriever with MMR
from langchain_community.vectorstores import InMemoryVectorStore
from langchain_community.embeddings.dashscope import DashScopeEmbeddings
from langchain_core.documents import Document
from dotenv import load_dotenv, find_dotenv

# init qwen model config
_ = load_dotenv(find_dotenv("./env/.env"))

embeddings = DashScopeEmbeddings(model="text-embedding-v3")
vectorStore = InMemoryVectorStore(embeddings)

document1 = Document(page_content="The powerhouse of the cell is the mitochondria", metadata={ "source": "https://example.com" })
document2 = Document(page_content="Buildings are made out of brick", metadata={ "source": "https://example.com" })
document3 = Document(page_content="Mitochondria are made out of lipids", metadata={ "source": "https://example.com" })
documents = [document1, document2, document3]

vectorStore.add_documents(documents)

simi_ret = vectorStore.similarity_search("biology", k=2)
print(simi_ret)

mmr_ret = vectorStore.as_retriever(search_type="mmr", search_kwargs={"k": 2, "lambda_mult": 0.5}).invoke("biology")
print(mmr_ret)

[Document(id='cd06846b-aab4-4417-bfac-204838fb3bc1', metadata={'source': 'https://example.com'}, page_content='The powerhouse of the cell is the mitochondria'), Document(id='d6e7bb82-9b9f-4ff7-8404-96e24300295b', metadata={'source': 'https://example.com'}, page_content='Mitochondria are made out of lipids')]
[Document(id='cd06846b-aab4-4417-bfac-204838fb3bc1', metadata={'source': 'https://example.com'}, page_content='The powerhouse of the cell is the mitochondria'), Document(id='9226e3c3-7c81-4f08-9909-3ba4e066c9db', metadata={'source': 'https://example.com'}, page_content='Buildings are made out of brick')]


In [5]:
%pip install --upgrade --quiet  lark langchain-chroma

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-ai-generativelanguage 0.6.6 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.3 which is incompatible.
langchain 0.3.0 requires langsmith<0.2.0,>=0.1.17, but you have langsmith 0.3.11 which is incompatible.
langchain-anthropic 0.1.23 requires langchain-core<0.3.0,>=0.2.26, but you have langchain-core 0.3.41 which is incompatible.
langchain-community 0.3.0 requires langsmith<0.2.0,>=0.1.112, but you have langsmith 0.3.11 which is incompatible.
langchain-experimental 0.0.64 requires langchain-community<0.3.0,>=0.2.10, but you have langchain-community 0.3.0 which is incompatible.
langchain-experimental 0.0.64 requires langchain-core<0.3.0,>=0.2.27, but you have langchain-core 0.3.41 which is incompatible.
langchain-google-gen

In [ ]:
from langchain_core.documents import Document
from langchain_community.vectorstores import InMemoryVectorStore
from langchain_community.vectorstores import DashVector
from langchain_community.embeddings.dashscope import DashScopeEmbeddings
from langchain.chains.query_constructor.schema import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_community.chat_models import ChatTongyi
from dotenv import load_dotenv, find_dotenv

# init qwen model config
_ = load_dotenv(find_dotenv("./env/.env"))

docs = [
    Document(
        page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
    ),
    Document(
        page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
    ),
    Document(
        page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6},
    ),
    Document(
        page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
    ),
    Document(
        page_content="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated"},
    ),
    Document(
        page_content="Three men walk into the Zone, three men walk out of the Zone",
        metadata={
            "year": 1979,
            "director": "Andrei Tarkovsky",
            "genre": "thriller",
            "rating": 9.9,
        },
    ),
]
embeddings = DashScopeEmbeddings(model="text-embedding-v3")
# in-memory vector store
vector_store = InMemoryVectorStore(embeddings)
vector_store.add_documents(documents)
# sqlite vector store with local storage
vector_store = SQLiteVSS.from_documents(
    documents,
    embeddings,
    table="movies",
    db_file="./vss-self-query.db",
)
# dashscope vector store service
vector_store = DashVector.from_documents(docs, embeddings)


metadata_field_info = [
    AttributeInfo(
        name="genre",
        description="The genre of the movie. One of ['science fiction', 'comedy', 'drama', 'thriller', 'romance', 'action', 'animated']",
        type="string",
    ),
    AttributeInfo(
        name="year",
        description="The year the movie was released",
        type="integer",
    ),
    AttributeInfo(
        name="director",
        description="The name of the movie director",
        type="string",
    ),
    AttributeInfo(
        name="rating", description="A 1-10 rating for the movie", type="float"
    ),
]
document_content_description = "Brief summary of a movie"
llm_model = "qwen-max"
llm = ChatTongyi(temperature=0.8, model=llm_model)
retriever = SelfQueryRetriever.from_llm(
    llm,
    vector_store,
    document_content_description,
    metadata_field_info,
)

# This example only specifies a filter
retriever.invoke("I want to watch a movie rated higher than 8.5")

# This example specifies a query and a filter
retriever.invoke("Has Greta Gerwig directed any movies about women")

# This example specifies a composite filter
retriever.invoke("What's a highly rated (above 8.5) science fiction film?")

# This example specifies a query and composite filter
retriever.invoke(
    "What's a movie after 1990 but before 2005 that's all about toys, and preferably is animated"
)

In [7]:
%pip install lark


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# self-query from scratch
from langchain.chains.query_constructor.base import (
    StructuredQueryOutputParser,
    get_query_constructor_prompt,
)
from langchain.chains.query_constructor.schema import AttributeInfo
from langchain_community.chat_models import ChatTongyi
from dotenv import load_dotenv, find_dotenv

# init qwen model config
_ = load_dotenv(find_dotenv("./env/.env"))

document_content_description = "Brief summary of a movie"
metadata_field_info = [
    AttributeInfo(
        name="genre",
        description="The genre of the movie. One of ['science fiction', 'comedy', 'drama', 'thriller', 'romance', 'action', 'animated']",
        type="string",
    ),
    AttributeInfo(
        name="year",
        description="The year the movie was released",
        type="integer",
    ),
    AttributeInfo(
        name="director",
        description="The name of the movie director",
        type="string",
    ),
    AttributeInfo(
        name="rating", description="A 1-10 rating for the movie", type="float"
    ),
]
llm_model = "qwen-max"
llm = ChatTongyi(temperature=0.8, model=llm_model)
prompt = get_query_constructor_prompt(
    document_content_description,
    metadata_field_info,
)
output_parser = StructuredQueryOutputParser.from_components()
query_constructor = prompt | llm | output_parser

print(prompt.format(query="dummy question"))

Your goal is to structure the user's query to match the request schema provided below.

<< Structured Request Schema >>
When responding use a markdown code snippet with a JSON object formatted in the following schema:

```json
{
    "query": string \ text string to compare to document contents
    "filter": string \ logical condition statement for filtering documents
}
```

The query string should contain only text that is expected to match the contents of documents. Any conditions in the filter should not be mentioned in the query as well.

A logical condition statement is composed of one or more comparison and logical operation statements.

A comparison statement takes the form: `comp(attr, val)`:
- `comp` (eq | ne | gt | gte | lt | lte | contain | like | in | nin): comparator
- `attr` (string):  name of attribute to apply the comparison to
- `val` (string): is the comparison value

A logical operation statement takes the form `op(statement1, statement2, ...)`:
- `op` (and | or | not

StructuredQuery(query='taxi drivers', filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='genre', value='science fiction'), Comparison(comparator=<Comparator.GTE: 'gte'>, attribute='year', value=1990), Comparison(comparator=<Comparator.LTE: 'lte'>, attribute='year', value=1999), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='director', value='Luc Besson')]), limit=None)

In [8]:
query_constructor.invoke(
    {
        "query": "What are some sci-fi movies from the 90's directed by Luc Besson about taxi drivers"
    }
)

StructuredQuery(query='taxi drivers', filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='genre', value='science fiction'), Comparison(comparator=<Comparator.GTE: 'gte'>, attribute='year', value=1990), Comparison(comparator=<Comparator.LTE: 'lte'>, attribute='year', value=1999), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='director', value='Luc Besson')]), limit=None)